In [11]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

In [13]:
df1 = pd.read_csv('/content/drive/MyDrive/congress_tweets.csv')
df2 = pd.read_csv('/content/drive/MyDrive/bjp_tweets.csv')
df = pd.concat([df1,df2])
df.head()

,Unnamed: 0,tweet,target
0,0,spending shocker cbo latest debt projection sh...,0
1,1,will you vote for congress in 2024 congress co...,1
2,2,triviapotus we need termlimits for congress,1
3,3,eam s jaishankar hit out at congress for quad ...,0
4,4,is this why congress supported amp celebrated ...,1


In [14]:
vocabulary_size = 5000

In [15]:
x = df["tweet"]
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.20)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

Loaded dataset with 63783 training samples, 15946 test samples


In [16]:
tokenizer = Tokenizer(num_words=1000,lower=True)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

In [17]:
from keras.utils.data_utils import pad_sequences

max_words = 500
X_train = pad_sequences(X_train, padding='post', maxlen=max_words)
X_test = pad_sequences(X_test, padding='post', maxlen=max_words)

In [18]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
batch_size = 64
num_epochs = 5

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Epoch 1/5
996/996 [==============================] - 889s 887ms/step - loss: 0.6913 - accuracy: 0.5314 - val_loss: 0.6841 - val_accuracy: 0.5781
Epoch 2/5
996/996 [==============================] - 896s 900ms/step - loss: 0.6912 - accuracy: 0.5321 - val_loss: 0.6853 - val_accuracy: 0.5781
Epoch 3/5
996/996 [==============================] - 880s 884ms/step - loss: 0.6911 - accuracy: 0.5321 - val_loss: 0.6870 - val_accuracy: 0.5781
Epoch 4/5
996/996 [==============================] - 862s 866ms/step - loss: 0.6912 - accuracy: 0.5321 - val_loss: 0.6847 - val_accuracy: 0.5781
Epoch 5/5
996/996 [==============================] - 857s 860ms/step - loss: 0.6911 - accuracy: 0.5321 - val_loss: 0.6858 - val_accuracy: 0.5781


In [ ]:
#Saving the model
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/models/pt5.h5')

In [ ]:
#Loading the model
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/models/pt5.h5')

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.538504958152771


In [ ]:
scores = model.evaluate(X_train, y_train, verbose=0)
print('Train accuracy:', scores[1])

Train accuracy: 0.532116711139679


In [ ]:
def analyze_sentiment(statement):
    statement_sequence = tokenizer.texts_to_sequences([statement])
    statement_padded = pad_sequences(statement_sequence, padding='post', maxlen=max_words)
    prediction = model.predict(statement_padded)[0][0]
    sentiment = 'Positive' if prediction >= 0.5 else 'Negative'
    return sentiment, prediction

# Example usage
statement = "The government's handling of the economic crisis has been a complete failure, with skyrocketing inflation and rising unemployment rates, leaving the common people to suffer while politicians enjoy their privileges"
sentiment, prediction = analyze_sentiment(statement)
print("Statement:", statement)
print("Sentiment:", sentiment)
print("Prediction score:", prediction)


1/1 [==============================] - 1s 510ms/step
Statement: The government's handling of the economic crisis has been a complete failure, with skyrocketing inflation and rising unemployment rates, leaving the common people to suffer while politicians enjoy their privileges
Sentiment: Positive
Prediction score: 0.5285933
